In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math


In [2]:
stocks = pd.read_csv('sp500.csv')


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url)
# type(data). show the data type

# see if show 404 response from http, 200 means ok
print(data.status_code)

data = requests.get(api_url).json()
#data
print(data['symbol'])

In [ ]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price)
print(market_cap)

In [4]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame([[0,0,0,0]], columns = my_columns)

print(final_dataframe)

   Ticker  Stock Price  Market Capitalization  Number of Shares to Buy
0       0            0                      0                        0


In [ ]:
tmpPD = pd.DataFrame([[
            symbol,
            price,
            market_cap,
            'N/A']
        ], columns=my_columns )

pd.concat([final_dataframe, tmpPD], ignore_index=True)

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = pd.concat([final_dataframe, 
        pd.DataFrame(
            [[
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ]],
            columns=my_columns
        )],
        ignore_index=True
    )

In [ ]:
final_dataframe


In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [6]:
symbol_groups = list(chunks(stocks['Ticker'], 100))

In [7]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

print(symbol_strings)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data.status_code)
    for symbol in symbol_string.split(','):
        final_dataframe = pd.concat( [final_dataframe,
                                        pd.DataFrame([[symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A']], 
                                                  columns = my_columns)], 
                                        ignore_index = True)

KeyError: 'DISCA'

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,134.56,40066829768,N/A
1,AAL,18.61,11871704297,N/A
2,AAP,194.17,11942517270,N/A
3,AAPL,155.76,2519900873827,N/A
4,ABBV,155,269566398579,N/A
...,...,...,...,...
130,DG,228.4,53030076995,N/A
131,DGX,150.66,17217372908,N/A
132,DHI,75.19,27225526816,N/A
133,DHR,276.29,202072408300,N/A


In [10]:
portfolio_size = input('Enter the value of your portfolio')
try:
    val = float(portfolio_size)
    print(val)
except:
    print('Please enter an integer')

200000.0


In [11]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])


In [12]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,Number Of Shares to Buy
0,A,134.56,40066829768,N/A,11.0
1,AAL,18.61,11871704297,N/A,79.0
2,AAP,194.17,11942517270,N/A,7.0
3,AAPL,155.76,2519900873827,N/A,9.0
4,ABBV,155,269566398579,N/A,9.0
...,...,...,...,...,...
130,DG,228.4,53030076995,N/A,6.0
131,DGX,150.66,17217372908,N/A,9.0
132,DHI,75.19,27225526816,N/A,19.0
133,DHR,276.29,202072408300,N/A,5.0


In [13]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

## Create Excel Format

In [14]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [15]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [16]:
writer.save()